In [26]:
import pandas as pd
import numpy as np
import requests
import geocoder

Use beautfiul soup to locate the table header and content

In [27]:
from bs4 import BeautifulSoup
page_address='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_content=requests.get(page_address).text
soup = BeautifulSoup(page_content,'html.parser')

sa=[]

#fetch column names
titlerow = soup.find('tr').text
head_=titlerow.split('\n')
del head_[0]
del head_[-1]

print(head_)

#fetch column content
for rows_ in soup.findAll('tr'):
    s=''
    for columns_ in rows_.findAll('td'):
        s=s+','+columns_.text

    s=s[1:]
    s=s.replace('\n','')
    sp=s.split(',')
    sa.append(sp)
  
#clean up      
del sa[0]
for i in range(293,288,-1):
    del sa[i]
    
print('Extracted from html...')

df = pd.DataFrame.from_records(sa, columns=head_)

['Postcode', 'Borough', 'Neighbourhood']
Extracted from html...


In [28]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [29]:
#get rid of Not assigned Boroughs
df1=df[df.Borough != 'Not assigned']

df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)

df1.sort_values(by=['Postcode','Borough'],inplace=True)

print('Created DataFrame...')

#rename nighbourhoods that have a not assigned value
for i in df1.index:
    if df1.iloc[i,2]=='Not assigned':
        df1.iloc[i,2]=df1.iloc[i,1]
    
print('Assigned neighborhood names...')

Created DataFrame...
Assigned neighborhood names...


/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/lib/python3.6/site-package

In [30]:
df1.head()

,Postcode,Borough,Neighbourhood
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern
21,M1C,Scarborough,Highland Creek
22,M1C,Scarborough,Rouge Hill
23,M1C,Scarborough,Port Union


Group by Postcode

In [31]:
#combine neighbourhoods with the same postalcode in one row
#find unique postal codes

dummy=df1['Postcode']

dummy.drop_duplicates(inplace=True)
df2=pd.DataFrame(dummy)
df2['Borough']='';
df2['Neighbourhood']='';


df2.reset_index(inplace=True)
df2.drop('index',axis=1,inplace=True)
df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)

#search for values in df1 and copy neighbourhood names to df2. Match criteris = Postcode
for i in df2.index:
    for j in df1.index:
        if df2.iloc[i,0]==df1.iloc[j,0]:
            df2.iloc[i,1]=df1.iloc[j,1]
            df2.iloc[i,2]=df2.iloc[i,2]+','+df1.iloc[j,2]
            
#get rid of extra ','
for i in df2.index:
    s=df2.iloc[i,2]
    if s[0]==',':
        s=s[1:]
    df2.iloc[i,2]=s

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/base.py:1241: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return self._update_inplace(result)
/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Check the grouping

In [32]:
print(df2.head(10))

  Postcode      Borough                                  Neighbourhood
0      M1B  Scarborough                                  Rouge,Malvern
1      M1C  Scarborough           Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough                Guildwood,Morningside,West Hill
3      M1G  Scarborough                                         Woburn
4      M1H  Scarborough                                      Cedarbrae
5      M1J  Scarborough                            Scarborough Village
6      M1K  Scarborough      East Birchmount Park,Ionview,Kennedy Park
7      M1L  Scarborough                  Clairlea,Golden Mile,Oakridge
8      M1M  Scarborough  Cliffcrest,Cliffside,Scarborough Village West
9      M1N  Scarborough                     Birch Cliff,Cliffside West


Check shape

In [10]:
print(df2.shape)

(103, 3)


In [60]:
df2['Latitude']='0';
df2['Longitude']='0';

gsc=pd.read_csv('Geospatial_Coordinates.csv')

Geocoder didnt work for me, so I am using the CSV file.

In [61]:
gsc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [62]:
for i in df2.index:
    for j in gsc.index:
        if df2.iloc[i,0]==gsc.iloc[j,0]:
            df2.iloc[i,3]=gsc.iloc[j,1]
            df2.iloc[i,4]=gsc.iloc[j,2]


Add Latitue and Longitude information to the dataframe. Matching criteria = postcode.

In [63]:
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [37]:
df2.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [54]:
df3=df2[df2.Borough.str.contains("Toronto")]

Getting rid of the boroughs that do not contain "Toronto"

In [74]:
df3.Borough.unique()
df3.head()
df3.shape

Running KMeans to cluster the neighborhoods into 4 clusters. Chances are they will cluster based on Borough(Downtown, East, West, Central)

In [96]:
import folium
toronto_map=folium.Map(location=[43.65, -79.4], zoom_start=11)


X=df3['Latitude']
Y=df3['Longitude']

x=np.asarray(X)
y=np.asarray(Y)

Z=np.stack((X,Y),axis=1)

from sklearn.cluster import KMeans 
k_means = KMeans(init = "k-means++", n_clusters = 4, n_init = 5)
k_means.fit(Z)
clusterindex=k_means.labels_


df3['cluster']=clusterindex

colors=['red','green','blue','yellow','pink','gray','magenta','cyan']


for lat, lng, label,cl_ind in zip(df3['Latitude'], df3['Longitude'], df3['Borough'],df3['cluster']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cl_ind], #set color according to clusterindex
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  

toronto_map



/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Indeed, the neighborhoods are clustered based on the Borough. Green is downtown, Yellow is East, Red is central and Blue is West.